In [2]:
pip install torchxrayvision

  Using cached scikit_image-0.25.2-cp312-cp312-win_amd64.whl.metadata (14 kB)
  Using cached imageio-2.37.0-py3-none-any.whl.metadata (5.2 kB)
  Using cached lazy_loader-0.4-py3-none-any.whl.metadata (7.6 kB)
   ---------------------------------------- 0.0/29.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/29.0 MB 640.0 kB/s eta 0:00:46
   ---------------------------------------- 0.1/29.0 MB 1.1 MB/s eta 0:00:28
   ---------------------------------------- 0.2/29.0 MB 1.8 MB/s eta 0:00:16
    --------------------------------------- 0.4/29.0 MB 2.7 MB/s eta 0:00:11
    --------------------------------------- 0.5/29.0 MB 2.2 MB/s eta 0:00:14
    --------------------------------------- 0.5/29.0 MB 2.0 MB/s eta 0:00:15
    --------------------------------------- 0.5/29.0 MB 1.6 MB/s eta 0:00:19
    --------------------------------------- 0.5/29.0 MB 1.3 MB/s eta 0:00:22
    --------------------------------------- 0.5/29.0 MB 1.2 MB/s eta 0:00:23
    ----------------------


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: E:\Python_DataScience\Agent_Systems\Ontology_Engineering\Scripts\python.exe -m pip install --upgrade pip


In [16]:
# --- Original Imports ---
import torch
from torchvision import models, transforms
from PIL import Image
import ollama # You already have this
import os
import numpy as np
import torchxrayvision as xrv
import rdflib
from rdflib import Graph, Literal, RDF, URIRef
from rdflib.namespace import RDFS


In [17]:
### NEW ###
# This cell loads your ontology file into an RDF graph

g = Graph()
g.parse("lung_ontology.ttl", format="turtle")

print(f"Ontology loaded successfully with {len(g)} statements.")

Ontology loaded successfully with 23 statements.


In [18]:
# ---------- Configuration ----------
OLLAMA_MODEL = "qwen:1.8b"
IMAGE_PATH = "sample_chest_xray.jpg" # Make sure this is a real X-ray image

# ---------- Image Preprocessing ----------
# ### MODIFIED ###
# This transform is correct. torchxrayvision models are compatible
# with standard ImageNet normalization, and your .convert("RGB")
# correctly handles the 3-channel input requirement.
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [19]:
# ---------- Image Preprocessing ----------
# ### MODIFIED ###
# We must use 1-channel (grayscale) normalization
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.5],  # 1-channel mean
        std=[0.5]    # 1-channel std
    )
])

In [20]:
def load_image(path):
    # ### MODIFIED ###
    # .convert("L") loads the image as 1-channel grayscale
    img = Image.open(path).convert("L")
    return transform(img).unsqueeze(0)

In [21]:
image_tensor = load_image(IMAGE_PATH)

In [22]:
image_tensor

tensor([[[[-0.8824, -0.8824, -0.8824,  ..., -0.8745, -0.8667, -0.8667],
          [-0.8745, -0.8745, -0.8745,  ..., -0.8745, -0.8667, -0.8667],
          [-0.8824, -0.8745, -0.8824,  ..., -0.8745, -0.8745, -0.8745],
          ...,
          [-0.8353, -0.8275, -0.8196,  ..., -0.8196, -0.8196, -0.8275],
          [-0.8353, -0.8275, -0.8275,  ..., -0.8196, -0.8196, -0.8275],
          [-0.8353, -0.8353, -0.8353,  ..., -0.8275, -0.8275, -0.8275]]]])

In [23]:
# ---------- Vision Model ----------
print("Analyzing chest X-ray using torchxrayvision (DenseNet)...")

# ### MODIFIED ###
# 1. Load the model trained on chest X-rays, not ImageNet
model = xrv.models.DenseNet(weights="densenet121-res224-all")
model.eval()

# ### MODIFIED ###
# 2. Get the *correct* list of diseases this model knows
# This REPLACES 'imagenet_labels'
disease_labels = model.pathologies
print(f"Model can detect: {disease_labels}")

# 3. Run the model
with torch.no_grad():
    outputs = model(image_tensor)
    
    # ### MODIFIED ###
    # This model is multi-label, so we apply a sigmoid
    # to get a probability (0.0 to 1.0) for each disease
    probabilities = torch.sigmoid(outputs[0])

# 4. Get the predictions
# We set a threshold (e.g., 0.5) to see what's "positive"
threshold = 0.5
top_predictions = []
for i, prob in enumerate(probabilities):
    if prob > threshold:
        top_predictions.append({
            "label": disease_labels[i],
            "confidence": float(prob)
        })

# If no disease is above the threshold, we assume 'Normal'
if not top_predictions:
    print("No pathology found above 0.5 threshold. Assuming Normal.")
    top_predictions.append({
        "label": "Normal", # We'll use this to query our ontology
        "confidence": 1.0 - float(torch.max(probabilities))
    })

print(f"Detected findings: {top_predictions}")

Analyzing chest X-ray using torchxrayvision (DenseNet)...
Model can detect: ['Atelectasis', 'Consolidation', 'Infiltration', 'Pneumothorax', 'Edema', 'Emphysema', 'Fibrosis', 'Effusion', 'Pneumonia', 'Pleural_Thickening', 'Cardiomegaly', 'Nodule', 'Mass', 'Hernia', 'Lung Lesion', 'Fracture', 'Lung Opacity', 'Enlarged Cardiomediastinum']
Detected findings: [{'label': 'Atelectasis', 'confidence': 0.6455538272857666}, {'label': 'Consolidation', 'confidence': 0.6402066946029663}, {'label': 'Infiltration', 'confidence': 0.6290329694747925}, {'label': 'Pneumothorax', 'confidence': 0.6338130831718445}, {'label': 'Edema', 'confidence': 0.6335951089859009}, {'label': 'Emphysema', 'confidence': 0.6283413171768188}, {'label': 'Fibrosis', 'confidence': 0.6315699219703674}, {'label': 'Effusion', 'confidence': 0.6629663109779358}, {'label': 'Pneumonia', 'confidence': 0.6347270011901855}, {'label': 'Pleural_Thickening', 'confidence': 0.643785834312439}, {'label': 'Cardiomegaly', 'confidence': 0.64844

In [24]:
# ### MODIFIED ###
# This summary is now a list of detected diseases
detected_diseases = [p['label'] for p in top_predictions]

# We will just use the first and most confident finding for our ontology query
if "Normal" in detected_diseases:
    disease_query = "Normal Lung" # Match the label in our ontology
else:
    disease_query = top_predictions[0]['label'] # e.g., "Pneumonia" or "Effusion"

print(f"Query for Ontology: {disease_query}")


Query for Ontology: Atelectasis


In [26]:
### NEW ###
print(f"Querying ontology for definition of '{disease_query}'...")

# This SPARQL query finds the 'rdfs:comment' (description) 
# for any node that has an 'rdfs:label' matching our disease_query.
sparql_query = f"""
PREFIX rdfs: <{RDFS}>

SELECT ?description
WHERE {{
    ?disease rdfs:label "{disease_query}" .
    ?disease rdfs:comment ?description .
}}
"""

# Execute the query
results = g.query(sparql_query)

factual_context = ""
for row in results:
    factual_context = str(row.description)
    break # We only need the first match

if not factual_context:
    factual_context = f"No specific definition found in the ontology for {disease_query}."


print("--- Retrieved Context from Ontology ---")
print(factual_context)
print("---------------------------------------")

Querying ontology for definition of 'Atelectasis'...
--- Retrieved Context from Ontology ---
Atelectasis is the collapse or closure of a part of the lung, resulting in reduced or absent gas exchange. On an X-ray, it may appear as a white, airless area or as a shift of the trachea or heart towards the collapsed area.
---------------------------------------


In [27]:
#---------- Prompt for Qwen ----------

# ### MODIFIED ###
# This new prompt FORCES Qwen to use your facts.
# This stops the hallucination.

prompt = f"""
You are a concise radiology assistant.
A vision model analyzed a chest X-ray and provided the following finding: {disease_query}

Use the following trusted context ONLY to write a brief summary.
DO NOT use any other information. DO NOT make things up.

TRUSTED CONTEXT:
"{factual_context}"

Based ONLY on the trusted context above, provide a one-sentence summary of the finding.
"""

print("--- Final Grounded Prompt ---")
print(prompt)
print("-----------------------------")

--- Final Grounded Prompt ---

You are a concise radiology assistant.
A vision model analyzed a chest X-ray and provided the following finding: Atelectasis

Use the following trusted context ONLY to write a brief summary.
DO NOT use any other information. DO NOT make things up.

TRUSTED CONTEXT:
"Atelectasis is the collapse or closure of a part of the lung, resulting in reduced or absent gas exchange. On an X-ray, it may appear as a white, airless area or as a shift of the trachea or heart towards the collapsed area."

Based ONLY on the trusted context above, provide a one-sentence summary of the finding.

-----------------------------


In [28]:

import ollama

def call_ollama(model_name: str, prompt_text: str) -> str:
    """Call a locally installed Ollama model and return its response."""""
    response = ollama.chat(
        model=model_name,
        messages=[{"role": "user", "content": prompt_text}]
    )
    return response['message']['content'].strip()

print("Generating interpretation with local Qwen-1.8B...")
report_text = call_ollama(OLLAMA_MODEL, prompt)

Generating interpretation with local Qwen-1.8B...


In [29]:

print(report_text)

Atelectasis is a type of lung collapse that reduces gas exchange and may lead to breathing difficulties or even death if left untreated.
